# ML Training Notebook

This notebook is designed to run on **Google Colab** with a **GPU runtime** (T4, A100, etc.).

## Quick setup
1. Go to **Runtime → Change runtime type → GPU**
2. Run the cells below in order
3. Monitor GPU usage with `!nvidia-smi`

## Cell 1: Check GPU

In [ ]:
!nvidia-smi

## Cell 2: Install dependencies from repo

In [ ]:
!pip -q install -r https://raw.githubusercontent.com/armanfeili/ml-colab-agentic/main/requirements.txt

## Cell 3: Quick sanity check with PyTorch

In [ ]:
import torch
import torchvision
from src.utils import set_seed, to_device

set_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

x = torch.randn(3, 3)
y = to_device(x)
print(f"Tensor device: {y.device}")

# Tiny training loop (toy example)
w = torch.randn(3, 3, requires_grad=True, device=device)
opt = torch.optim.SGD([w], lr=0.1)

for epoch in range(5):
    loss = (w @ w.T).mean()
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(f"Epoch {epoch}: loss={float(loss):.4f}")

print("\n✅ Toy training loop completed!")

## Cell 4: Create outputs folder and write metrics

In [ ]:
import os
import pathlib

pathlib.Path("outputs").mkdir(exist_ok=True)

with open("outputs/metrics.csv", "w") as f:
    f.write("epoch,loss\n")
    for i in range(5):
        f.write(f"{i},0.{i}\n")

print("✅ Wrote outputs/metrics.csv")

# List outputs
!ls -la outputs/